In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

In [3]:
df_movie=pd.read_csv('ratings.csv')

In [4]:
df_movie

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [5]:
Utility_Maxtrix=df_movie.pivot_table(index='movieId',columns='userId',values='rating')

In [6]:
Utility_Maxtrix

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,NaN,NaN,4.0,NaN,4.5,NaN,NaN,NaN,...,4.0,NaN,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,...,NaN,4.0,NaN,5.0,3.5,NaN,NaN,2.0,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
Utility_Maxtrix=np.array(Utility_Maxtrix)

In [8]:
def Normalized_Maxtrix(arr):
    arr_temp=np.column_stack(arr)
    Avg_columns = np.zeros(arr_temp.shape[0])
    for i in range(arr_temp.shape[0]):
        Avg_columns[i] = round(np.nanmean(arr_temp[i]),2)
    for i in range(len(arr)):
        for j in range(len(arr[0])):
            arr[i][j]= arr[i][j] - Avg_columns[j]
            if np.isnan(arr[i][j]):
                arr[i][j]=0
    A_sparse=sparse.csr_matrix(arr)
    return A_sparse

In [27]:
example_rating = np.array([[2,np.nan,np.nan,5,1,np.nan]  ### tesst
                           ,[np.nan,np.nan,3,4,3,1]
                           ,[3,1,3,np.nan,5,np.nan],
                           [np.nan,4,np.nan,np.nan,5,3]
                           ,[np.nan,1,2,3,np.nan,np.nan]
                           ,[1,np.nan,2,np.nan,2,np.nan]])

Avg_col=Normalized_Maxtrix(example_rating)
Avg_col.toarray()

array([[ 0. ,  0. ,  0. ,  1. , -2.2,  0. ],
       [ 0. ,  0. ,  0.5,  0. , -0.2, -1. ],
       [ 1. , -1. ,  0.5,  0. ,  1.8,  0. ],
       [ 0. ,  2. ,  0. ,  0. ,  1.8,  1. ],
       [ 0. , -1. , -0.5, -1. ,  0. ,  0. ],
       [-1. ,  0. , -0.5,  0. , -1.2,  0. ]])

In [16]:
Avg_col=Normalized_Maxtrix(Utility_Maxtrix)

In [21]:
Avg_col.toarray()

array([[-0.37,  0.  ,  0.  , ..., -0.63, -0.27,  1.31],
       [ 0.  ,  0.  ,  0.  , ..., -1.13,  0.  ,  0.  ],
       [-0.37,  0.  ,  0.  , ..., -1.13,  0.  ,  0.  ],
       ...,
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ]])

In [22]:
# Hàm tính norm của một hàng
def row_norm(row):
    return np.sqrt(np.sum(row.data ** 2))

# Hàm tính dot product giữa hai hàng
def dot_product(a, b):
    indices1 = a.indices
    indices2 = b.indices
    common_indices = np.intersect1d(indices1, indices2, assume_unique=True)

    product_sum = 0.0
    for idx in common_indices:
        product_sum += a[0, idx] * b[0, idx]

    return product_sum

# Hàm tính ma trận cosine similarity
def cosine_similarity_sparse_matrix(sparse_matrix):
    num_rows = sparse_matrix.shape[0]

    # Tính norm của từng hàng
    norms = np.zeros(num_rows)
    for i in range(num_rows):
        norms[i] = row_norm(sparse_matrix[i])

    # Tính ma trận dot product
    dot_product_matrix = np.zeros((num_rows, num_rows))
    for i in range(num_rows):
        for j in range(num_rows):
            dot_product_matrix[i, j] = dot_product(sparse_matrix[i], sparse_matrix[j])

    # Tính ma trận cosine similarity
    similarity_matrix = np.zeros((num_rows, num_rows))
    for i in range(num_rows):
        for j in range(num_rows):
            if norms[i] != 0 and norms[j] != 0:
                similarity_matrix[i, j] = dot_product_matrix[i, j] / (norms[i] * norms[j])
            else:
                similarity_matrix[i, j] = 0.0

    return similarity_matrix

In [26]:
similarity_matrix = cosine_similarity_sparse_matrix(Avg_col)
similarity_matrix

array([[ 1.        ,  0.1603066 , -0.6993629 , -0.57085414, -0.27586863,
         0.66607229],
       [ 0.1603066 ,  1.        , -0.0413344 , -0.41713867, -0.14674182,
        -0.0053682 ],
       [-0.6993629 , -0.0413344 ,  1.        ,  0.18436221,  0.2133948 ,
        -0.8873441 ],
       [-0.57085414, -0.41713867,  0.18436221,  1.        , -0.46448868,
        -0.45879007],
       [-0.27586863, -0.14674182,  0.2133948 , -0.46448868,  1.        ,
         0.10161846],
       [ 0.66607229, -0.0053682 , -0.8873441 , -0.45879007,  0.10161846,
         1.        ]])